In [1]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import transformers
import mlflow
import importlib

import sys
sys.path.append("../")
from src.data import datasets
importlib.reload(datasets)
from src import models
importlib.reload(models)
importlib.reload(models.lstm_models)
importlib.reload(models.train_model)

c:\Users\tatur\anaconda3\envs\maslab_base\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'src.models.train_model' from 'd:\\etu\\nn\\etu_recur_paraident\\etu_paraphrase_ident\\notebooks\\..\\src\\models\\train_model.py'>

In [2]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")

Using CUDA


In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained("microsoft/deberta-v3-large", use_fast=True)
data_dir = r'D:\etu\nn\etu_recur_paraident\etu_paraphrase_ident\data\processed'
TRAIN = 'train'
VAL = 'val'
# Создание датасетов на основе torch ImageFolder
text_datasets = {
    x: datasets.ParaphraseDataset(
        pd.read_csv(data_dir + "\\" + x + ".csv"),
        tokenizer,
        256
    )
    for x in [TRAIN, VAL]
}

dataloaders = {
    x: torch.utils.data.DataLoader(
        text_datasets[x], batch_size=16,
        shuffle=True,# num_workers=8,
      #  collate_fn = collate_fn
    )
    for x in [TRAIN, VAL]
}

c:\Users\tatur\anaconda3\envs\maslab_base\lib\site-packages\transformers\convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [4]:
next(iter(dataloaders["train"]))["input_ids"].shape

torch.Size([16, 1, 256])

In [5]:
import json
file = open(r"D:\etu\nn\etu_recur_paraident\etu_paraphrase_ident\models\lstm0.json", "r")
d = json.load(file)
file.close()
d

{'vocab_size': 128000,
 'embedding_dim': 100,
 'lstm': {'hidden_size': 150, 'num_layers': 3, 'dropout': 0.2},
 'linear': {'num_layers': 3, 'include_bn': True, 'layers': [512, 256, 128]},
 'out_classes': 2}

In [6]:
model = models.lstm_models.build_SimpleBiLSTM(d)
print(model)

LSTMModel(
  (embedding): Embedding(128000, 100)
  (lstm): LSTM(100, 150, num_layers=3, batch_first=True, dropout=0.2, bidirectional=True)
  (linear): Sequential(
    (0): Linear(in_features=300, out_features=512, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): ReLU()
    (5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=256, out_features=128, bias=True)
    (7): ReLU()
    (8): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Linear(in_features=128, out_features=2, bias=True)
    (10): Softmax(dim=None)
  )
)


In [7]:
models.train_model.train_model(model, torch.nn.BCELoss(), optim.Adam(model.parameters()), dataloaders, scheduler=None)

Epoch 0/10
----------
Training batch 0/17688

c:\Users\tatur\anaconda3\envs\maslab_base\lib\site-packages\torch\nn\modules\module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


ValueError: Using a target size (torch.Size([16, 1])) that is different to the input size (torch.Size([16, 2])) is deprecated. Please ensure they have the same size.